# Examples for Using HHL

## Simple Example to Validate Result

Let's first try a simple example and validate that our code gives the correct results. Remember that the answer in the HHL algorithm is not classical, but encoded in a statevector, which is difficult to directly observe. In order to validate our result, we use the `Statevector` class from the `quantum_info` package to obtain the exact vector components.

In [2]:
import numpy as np

from hhl import HHL
from qiskit.quantum_info import Statevector

A = np.array([[1, -1/3], [-1/3, 1]])
b = np.array([1, 0])

qc = HHL(A, b)

print("quantum circuit: ")
print(qc)

quantum_answer = Statevector(qc).data.real[16:18] / qc.scaling # 16 = 10000; 17 = 10001
classic_answer = np.linalg.solve(A,b)

np.set_printoptions(3)

print("quantum answer = ", quantum_answer)
print("classic answer = ", classic_answer)

quantum circuit: 
      ┌─────────────┐┌──────┐        ┌─────────┐
  q1: ┤ circuit-167 ├┤3     ├────────┤3        ├
      └─────────────┘│      │┌──────┐│         │
q2_0: ───────────────┤0     ├┤2     ├┤0        ├
                     │  QPE ││      ││  QPE_dg │
q2_1: ───────────────┤1     ├┤1     ├┤1        ├
                     │      ││  1/x ││         │
q2_2: ───────────────┤2     ├┤0     ├┤2        ├
                     └──────┘│      │└─────────┘
  q3: ───────────────────────┤3     ├───────────
                             └──────┘           
quantum answer =  [1.125 0.375]
classic answer =  [1.125 0.375]


## Analyse Circuit Complexity

Let's first look empirically how deep is our circuit.

In [3]:
from qiskit import transpile
from scipy.sparse import diags
MAX_QUBITS = 4

depths = []
for n_qubits in range(1, MAX_QUBITS+1):
    # construct sparser matrix
    A_ = diags([-1/3, 1, -1/3],
                   [-1, 0, 1],
                   shape=(2**n_qubits, 2**n_qubits)).toarray()
    b_ = np.array([1] + [0]*(2**n_qubits -1))
    
    qc = HHL(A_, b_)
    qc = transpile(qc, basis_gates=['id', 'rz', 'sx', 'x', 'cx'])
    depths.append(qc.depth())

print(depths)

[334, 4335, 36343, 225024]


## Run on Real Hardware



In [10]:
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2 as Sampler
from qiskit.visualization import plot_histogram
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

qc.measure_all()

service = QiskitRuntimeService()
backend = service.least_busy(operational=True, simulator=False, min_num_qubits=qc.num_qubits)

pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
isa_circuit = pm.run(qc)

sampler = Sampler(backend)
job = sampler.run([isa_circuit])
result = job.result()
 
# Get results for the first (and only) PUB
pub_result = result[0]
 
print(f" > Counts: {pub_result.data.meas.get_counts()}")